In [3]:
import pathlib
import pandas as pd
import numpy as np
import os
from BlockSync_current import *

In [4]:
# ----------------------------------------------------------------------------------
# Organiser-workbook generator  |  This works, but pay attention to recNames
# ----------------------------------------------------------------------------------
from __future__ import annotations
import re, datetime, pathlib, pandas as pd

# ────────────────────────────────────────────────────────────────────────────────
def map_windows_to_unix(win_path: str, mapping: dict[str,str]) -> str:
    """
    Replace the Windows prefix in win_path with the UNIX prefix, and turn backslashes
    into forward slashes.  If no prefix matches, still convert to forward slashes.
    """
    for win_prefix, unix_prefix in mapping.items():
        if win_path.lower().startswith(win_prefix.lower()):
            rel = win_path[len(win_prefix):]
            return unix_prefix.rstrip("/") + rel.replace("\\", "/")
    return win_path.replace("\\", "/")


# ────────────────────────────────────────────────────────────────────────────────
# 1.  helper – crawl PV_xx tree and collect “Record Node …” folders
# ────────────────────────────────────────────────────────────────────────────────
def get_paths_to_record_nodes(tree_root_path: pathlib.Path) -> list[pathlib.Path]:
    def match_date(s: str)  -> bool: return re.fullmatch(r"\d{4}_\d{2}_\d{2}", s) is not None
    def match_block(s: str) -> bool: return re.fullmatch(r"block_\d{3}", s)    is not None

    paths = []
    for date_dir in filter(lambda p: p.is_dir() and match_date(p.name), tree_root_path.iterdir()):
        for block in filter(lambda p: p.is_dir() and match_block(p.name), date_dir.iterdir()):
            oe_path      = block / "oe_files"
            datetime_dir = next(oe_path.iterdir())                     # only one subfolder
            rec_node_dir = next(i for i in datetime_dir.iterdir() if "Node" in i.name)
            paths.append(rec_node_dir.resolve())
    return paths


# ────────────────────────────────────────────────────────────────────────────────
# 2.  main – build organiser DataFrame and write .xlsx
# ────────────────────────────────────────────────────────────────────────────────
def build_organiser_excel(
        animal_root   : str | pathlib.Path,
        export_dir    : str | pathlib.Path,
        template_xlsx : str | pathlib.Path | None = None,
        path_map      : dict[str,str] = None,       # ← new!
) -> pathlib.Path:
    """
    Scan *animal_root* and create an organiser workbook in *export_dir*.
    If *path_map* is provided, windows paths will be remapped to UNIX.
    """
    animal_root = pathlib.Path(animal_root).expanduser().resolve()
    export_dir  = pathlib.Path(export_dir ).expanduser().resolve()
    export_dir.mkdir(parents=True, exist_ok=True)

    # discover record nodes
    rec_nodes = sorted(get_paths_to_record_nodes(animal_root))
    if not rec_nodes:
        raise RuntimeError(f"No record-node folders found under {animal_root}")

    # column layout + defaults (unchanged) …
    if template_xlsx:
        tpl      = pd.read_excel(template_xlsx, engine="openpyxl")
        columns  = list(tpl.columns)
        defaults = {c: tpl[c].dropna().iloc[0] if tpl[c].notna().any() else pd.NA for c in columns}
    else:
        columns = [
            'Unnamed: 0','Exclude','spikes','sortedManually','videoSync','Breathing',
            'Animal','recNames','Remarks','Date','MEAfiles','recFormat','Sex','Species',
            'MEA_Layout','folder','VideoFiles','temperatureFile','Video_notes',
            'LFPCh_verified','defaulLFPCh','Temp','tempMedian','accelerometerCh',
            'Complete_recording','TempLogger_file','Temp_verifcation','video_start_time',
            'video_end_time','AUX_data','video_triggers'
        ]
        defaults = dict.fromkeys(columns, pd.NA)
        defaults.update({
            'recFormat': 'OERecording',
            'Sex'      : 'M',
            'Species'  : 'Pogona_Vitticeps',
            'MEA_Layout': 'layout_120_32x1_H4_CamNeuro',
            'defaulLFPCh': 15,
            'Temp'       : 27,
            'accelerometerCh': '1,2,3',
        })

    # one row per record node
    rows = []
    for idx, rec_path in enumerate(rec_nodes):
        date_folder  = rec_path.parents[4].name
        block_folder = rec_path.parents[3].name
        block_num    = int(block_folder.split('_')[1])
        rec_name     = f"Block{block_num:04d}"

        row = {c: defaults.get(c, pd.NA) for c in columns}
        row.update({
            'Unnamed: 0': idx,
            'Animal'    : animal_root.name,
            'Date'      : date_folder,
            'recNames'  : rec_name,
        })

        # ← here’s the change: map or just normalize slashes
        orig = str(rec_path)
        if path_map:
            folder = map_windows_to_unix(orig, path_map)
        else:
            folder = orig.replace("\\", "/")
        row['folder'] = folder

        rows.append(row)

    organiser = pd.DataFrame(rows, columns=columns)

    # write workbook
    outfile = export_dir / f"{animal_root.name}_organiser_{datetime.datetime.now():%Y%m%d_%H%M}.xlsx"
    organiser.to_excel(outfile, index=False)
    print(f"✔ organiser written to {outfile}")
    return outfile


# ────────────────────────────────────────────────────────────────────────────────
# 3.  quick test --- edit these two paths and re-run
# ────────────────────────────────────────────────────────────────────────────────
animal_folder   = r"X:\Nimrod\experiments\PV_143"
output_folder   = r"X:\Nimrod\experiments\PV_143\HelperFiles"
template_file   = r"X:\Nimrod\experiments\PV_106\HelperFiles\PV_106_monolith_old.xlsx"

# define your Windows→Linux mapping here:
win_to_unix = {
    r"X:\Nimrod\experiments": r"/media/sil1/Data/Nimrod/experiments",
    r"Z:\Nimrod\experiments": r"/media/sil2/Data/Nimrod/experiments"
}

# then call:
build_organiser_excel(animal_folder, output_folder, template_file, path_map=win_to_unix)


✔ organiser written to \\sil1\data\Nimrod\experiments\PV_143\HelperFiles\PV_143_organiser_20250812_1333.xlsx


WindowsPath('//sil1/data/Nimrod/experiments/PV_143/HelperFiles/PV_143_organiser_20250812_1333.xlsx')

In [2]:
# arrange all recordings according to csv format for matlab analysis
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24\sortedBlocks.xlsx')
df_dict = {
    'Exclude':'',
    'spikes': '',
    'sortedManually': '',
    'videoSync':'',
    'Breathing': '',
    'Animal': '',
    'recNames': '',
    'Remarks': '',
    'Date': '',
    'MEAfiles':'',
    'recFormat': '',
    'Sex': '',
    'Species':'',
    'MEA_Layout':'',
    'folder': '',
    'VideoFiles':'',
    'temperatureFile':'',
    'Video_notes':'',
    'LFPCh_verified':'',
    'defaulLFPCh':'',
    'Temp':'',
    'tempMedian':'',
    'accelerometerCh':'',
    'Complete_recording':'',
    'TempLogger_file':'',
    'Temp_verifcation':'',
    'video_start_time':'',
    'video_end_time':'',
    'AUX_data':'',
    'video_triggers':''
}
df = pd.DataFrame(columns=df_dict.keys())
df

,Exclude,spikes,sortedManually,videoSync,Breathing,Animal,recNames,Remarks,Date,MEAfiles,...,Temp,tempMedian,accelerometerCh,Complete_recording,TempLogger_file,Temp_verifcation,video_start_time,video_end_time,AUX_data,video_triggers


In [2]:
# collect blocks into the dataframe:

# define the path to the experiment folder:
path_to_experiments = pathlib.Path(r"Z:\Nimrod\experiments")
animal = "PV_106"
p = path_to_experiments / animal
# collect dates
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
print(date_folder_list)
for date in date_folder_list:
    dp = p / animal / date

    # list all blocks in the folder:
    blocks = [i for i in dp.iterdir() if 'block' in str(i).lower() and i.is_dir()]
    for block in blocks:
        bp = dp / block
        blocksync = BlockSync(animal_call=animal,
                      experiment_date=date.name,
                      block_num=block.name[6:],
                      path_to_animal_folder=str(path_to_experiments))
        df_row = {
            'Exclude':'',
            'spikes': '',
            'sortedManually': '',
            'videoSync':'',
            'Breathing': '',
            'Animal': animal,
            'recNames': f'Block0{block.name[6:]}',
            'Remarks': '',
            'Date': blocksync.experiment_date,
            'MEAfiles':'',
            'recFormat': 'OERecording',
            'Sex': 'M',
            'Species':'Pogona_Vitticeps',
            'MEA_Layout':'layout_120_32x1_H4_CamNeuro',
            'folder': '/media/sil2/Data' + str(blocksync.oe_path.as_posix())[2:],
            'VideoFiles':'',
            'temperatureFile':'',
            'Video_notes':'',
            'LFPCh_verified':'',
            'defaulLFPCh':'15',
            'Temp':'27',
            'tempMedian':'',
            'accelerometerCh':'33,34,35',
            'Complete_recording':'',
            'TempLogger_file':'',
            'Temp_verifcation':'',
            'video_start_time':'',
            'video_end_time':'',
            'AUX_data':'',
            'video_triggers':''
        }
        df = df.append(df_row,ignore_index=True)

[WindowsPath('Z:/Nimrod/experiments/PV_106/2025_07_28'), WindowsPath('Z:/Nimrod/experiments/PV_106/2025_07_29'), WindowsPath('Z:/Nimrod/experiments/PV_106/unsorted_OE')]
instantiated block number 001 at Path: Z:\Nimrod\experiments\PV_106\2025_07_28\block_001, new OE version
Found the sample rate for block 001 in the xml file, it is 20000 Hz
No open ephys record node here!!!
retrieving zertoh sample number for block 001


AttributeError: 'BlockSync' object has no attribute 'oe_rec'

In [3]:

block = BlockSync(animal_call=animal,
                  experiment_date='2022_01_02',
                  block_num='038',
                  path_to_animal_folder=str(path_to_experiments))
#block.oe_path
print(block.experiment_date)
#block.experiment_date[8:] + block.experiment_date[5:7] + block.experiment_date[2:4]

NameError: name 'animal' is not defined

In [4]:
export_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_62\automated_block_collection_ubuntu.csv')
csv_path = export_path

In [33]:
import pandas as pd
from pathlib import Path

# 1) Linux→Windows helper (for reading)
def linux_to_win(linux_path: str,
                 prefix: str = '/media/sil2/Data',
                 win_drive: str = 'Z:') -> Path:
    if linux_path.startswith(prefix):
        rest = linux_path[len(prefix):].lstrip('/')
        return Path(f"{win_drive}\\{rest}")
    return Path(linux_path)

# 2) Windows→Linux helper (for writing)
def win_to_linux(win_path: Path,
                 win_drive: str = 'Z:',
                 linux_prefix: str = '/media/sil2/Data') -> str:
    # only remap if on the expected drive
    if win_path.drive.lower() == win_drive.lower():
        anchor = win_path.anchor         # e.g. "Z:\\"
        rel = win_path.relative_to(anchor)  # e.g. Path("PV_62/2023_04_22/…")
        return f"{linux_prefix}/{rel.as_posix()}"
    # otherwise just give POSIX version of whatever path we found
    return win_path.as_posix()

def fill_arena_video_paths(excel_input: str,
                           excel_output: str = None,
                           sheet_name=0):
    df = pd.read_excel(excel_input, sheet_name=sheet_name, engine='openpyxl')
    video_exts = {'.mp4', '.h264'}

    for idx, row in df.iterrows():
        # remap folder field into a Windows Path we can actually walk
        oe_path = linux_to_win(row['folder'])

        # find the block_xxx ancestor
        block_dir = next(
            (p for p in oe_path.parents if p.name.lower().startswith('block_')),
            None
        )
        if block_dir is None:
            print(f"[WARN] no block_xxx for row {idx}: {oe_path}")
            continue

        # try to find arena_videos subfolder
        try:
            arena_dirs = [
                d for d in block_dir.iterdir()
                if d.is_dir() and d.name.lower() == 'arena_videos'
            ]
        except FileNotFoundError:
            print(f"[WARN] {block_dir} missing on disk, skipping row {idx}")
            continue

        if not arena_dirs:
            print(f"[WARN] no arena_videos under {block_dir}, skipping row {idx}")
            continue

        arena_root = arena_dirs[0]

        # recursive collect any .mp4/.h264 (case-insensitive)
        matches = [
            f for f in arena_root.rglob('*')
            if f.is_file() and f.suffix.lower() in video_exts
        ]

        if matches:
            # remap back to Linux-style before writing
            linux_path = win_to_linux(matches[0])
            df.at[idx, 'VideoFiles'] = linux_path
        else:
            print(f"[INFO] no video files under {arena_root}, row {idx}")
            df.at[idx, 'VideoFiles'] = ''

    out = excel_output or excel_input
    df.to_excel(out, index=False, engine='openpyxl')
    print(f"✅ Written updated Excel to {out!r}")



In [35]:
# csv_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_62\pv_62_sleep_analysis_monolith.xlsx')
# output_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_62\pv_62_sleep_analysis_monolith_filled.xlsx')
csv_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_126\pv126_xl_for_mark_analysis.xlsx')
output_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_126\pv126_xl_for_mark_analysis_filled.xlsx')

fill_arena_video_paths(csv_path,output_path)

✅ Written updated Excel to WindowsPath('Z:/Nimrod/experiments/PV_126/pv126_xl_for_mark_analysis_filled.xlsx')


In [13]:
!pip install openpyxl